# Relaxation Mode

In [1]:
from saeno.FiniteBodyForces import FiniteBodyForces
    
# initialize the object
M = FiniteBodyForces()

In [2]:
from saeno.materials import semiAffineFiberMaterial

# provide a material model
material = semiAffineFiberMaterial(1645, 0.0008, 0.0075, 0.033)
M.setMaterialModel(material)

In [3]:
import numpy as np

# define the coordinates of the nodes of the mesh
# the array has to have the shape N_v x 3
R = np.array([[0., 0., 0.],  # 0
              [0., 1., 0.],  # 1
              [1., 1., 0.],  # 2
              [1., 0., 0.],  # 3
              [0., 0., 1.],  # 4
              [1., 0., 1.],  # 5
              [1., 1., 1.],  # 6
              [0., 1., 1.]]) # 7

# define the tetrahedra of the mesh
# the array has to have the shape N_t x 4
# every entry is an index referencing a verces in R (indices start with 0)
T = np.array([[0, 1, 7, 2],
              [0, 2, 5, 3],
              [0, 4, 5, 7],
              [2, 5, 6, 7],
              [0, 7, 5, 2]])

# define if the nodes are "variable", e.g. allowed to be moved by the solver
# a boolean matrix with shape N_v
#                   0      1      2      3      4      5      6      7
var = np.array([ True,  True, False, False,  True, False, False,  True])

# the initial displacements of the nodes
# if the node is fixed (e.g. not variable) than this displacement will be fixed
# during the solving
U = np.array([[ 0.  ,  0.  ,  0.  ],  # 0
              [ 0.  ,  0.  ,  0.  ],  # 1
              [-0.25,  0.  ,  0.  ],  # 2
              [-0.25,  0.  ,  0.  ],  # 3
              [ 0.  ,  0.  ,  0.  ],  # 4
              [-0.25,  0.  ,  0.  ],  # 5
              [-0.25,  0.  ,  0.  ],  # 6
              [ 0.  ,  0.  ,  0.  ]]) # 7

In [4]:
# provide the node data
M.setNodes(R, var, U)
# and the tetrahedron data
M.setTetrahedra(T)

In [5]:
# relax the mesh and move the "varible" nodes
M.relax()

updating forces and stiffness matrix finished 3.19s
updating forces and stiffness matrix finished 0.06s
Newton  0 : du= 0.031156870127166923   Energy= 100000000000.0   Residuum= 1.0327892029427537e+22
updating forces and stiffness matrix finished 0.06s
Newton  1 : du= 0.0292415021637474   Energy= 100000000000.0   Residuum= 9.748154637951108e+21
updating forces and stiffness matrix finished 0.06s
Newton  2 : du= 0.026997081132468067   Energy= 100000000000.0   Residuum= 9.138863858833917e+21
updating forces and stiffness matrix finished 0.06s
Newton  3 : du= 0.024434046967636177   Energy= 100000000000.0   Residuum= 8.504454079265089e+21
updating forces and stiffness matrix finished 0.06s
Newton  4 : du= 0.021604060891893886   Energy= 100000000000.0   Residuum= 7.848448698197649e+21
updating forces and stiffness matrix finished 0.06s
Newton  5 : du= 0.018594624755017816   Energy= 100000000000.0   Residuum= 7.177952409714575e+21
updating forces and stiffness matrix finished 0.07s
Newton  6

In [6]:
# store the forces of the nodes
M.storeF("F.dat")
# store the positions and the displacements
M.storeRAndU("R.dat", "U.dat")
# store the center of each tetrahedron and a combined list with energies and volumina of the tetrahedrons
M.storeEandV("RR.dat", "EV.dat")

F.dat stored.
R.dat stored.
U.dat stored.
RR.dat stored.
EV.dat stored.
